# Predict images with original paper models

This paper use the clone of the Github Repo of the original paper to reproduce all the results of the paper.

In [ ]:
# dataset import
from google.colab import drive
drive.mount('/content/drive')

import os
import csv
import glob
import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import display
from sklearn.metrics import accuracy_score

Mounted at /content/drive


In [ ]:
BASE_DIR = "/content/drive/MyDrive/Thesis/"
IMG_DIR = BASE_DIR + "dataset/t4sa/"
LABELING_DIR = IMG_DIR + "labeling/"

NEW_DATASET_DIR = BASE_DIR + "dataset/t4sa/new_text_predictions/bt4sa/"
ORIGINAL_DATASET_DIR = BASE_DIR + "dataset/t4sa/original_text_predictions/"

BENCHMARK_DIR = BASE_DIR + "dataset/benchmark/Twitter1269/"
BENCHMARK_DATASET_DIR = BENCHMARK_DIR + "twitter1/"

NEW_PREDICTION_DIR = BASE_DIR + "predictions/b-t4sa/new_predictions/"
ORIGINAL_PREDICTION_DIR = BASE_DIR + "predictions/b-t4sa/original_predictions/"
BENCHMARK_PREDICTION_DIR = BASE_DIR + "predictions/twitter1/"

# Predict with Previous model

In [ ]:
# Utilities

def get_results(pred_dir, benchmark, to_binary=False):
  '''Returns a dataframe with the results obtained by all the predictions in the directory on the benchmark specified.'''
  all_accuracy = [] 
  models = [name for name in os.listdir(pred_dir) if not os.path.isdir(pred_dir + name)]

  #pred_annot = pd.read_csv(label_file, header=None, names=["path", "label"],  delimiter='\t')
  #pred_annot.drop(pred_annot.index[pred_annot['path'] == "data/78439/784393626990194689-4.jpg"], inplace=True)

  pred_annot = pd.read_csv(benchmark)
  gold_labels = pred_annot['label'].tolist()
  
  # transform the gold label of the benchmark to get the same integer id for the positive class (2 instead of 1)
  if to_binary:
    gold_labels = [elem if elem == 0 else 2 for elem in gold_labels]

  # print the distribution of the gold labels
  unique, counts = np.unique(gold_labels, return_counts=True)
  print("Gold labels", dict(zip(unique, counts)))

  for model in models:
    predictions = pd.read_csv(pred_dir + model, header=None,  delimiter=',')

    predictions.pop(1)                          # remove the neutral class since the benchmark is a binary classification problem
    pred_labels = predictions.idxmax(axis = 1)  # get the maximum value as label prediction
    pred_labels = pred_labels.to_list()

    curr_accuracy = accuracy_score(pred_labels, gold_labels)   # compute the accuracy
    
    all_accuracy.append(curr_accuracy)

    # print the distribution of the predicted labels
    unique, counts = np.unique(pred_labels, return_counts=True)
    print(f"Labels predicted by {model}:", dict(zip(unique, counts)))

  raw_data = {'Model': models, 'Accuracy': all_accuracy}
  df = pd.DataFrame(raw_data)
  return df

In [ ]:
# Create the text file with the path of all the images to predict 
test_set = pd.read_csv(BENCHMARK_DIR + "5agree.csv")
display(test_set)
test_set = test_set['path'].to_list()

#test_set = [elem.split("/")[-1] for elem in test_set]
#test_set.remove("data/78439/784393626990194689-4.jpg")  #corrupted image

with open("test_set.txt", "w") as fobj:
  for x in test_set:
    fobj.write(x + "\n")

,path,label
0,1000028444_428246659240579072_BfFq8nsCEAAD4Ao.jpg,1
1,100003032_428124307202400256_Be_S6KkCMAA411d.jpg,1
2,100003356_426788043899412481_BZex9B2CQAAGAG4.jpg,1
3,100009310_428639141245767680_BfLVBxsIIAAkO7h.jpg,1
4,100010501_428500993441935360_BfIdnl4CMAANo3i.jpg,1
...,...,...
877,9766942_467674630996492289_Bn2DmPnIEAA08HQ.jpg,1
878,987839887_467330379293417472_Bno_GhIIcAEMqST.jpg,1
879,994742521_463943357186600960_BW-nugdCIAEyTNL.jpg,1
880,99579876_466234755802230785_Bng8fs-IgAAVsVp.jpg,0


In [ ]:
# copy all the converted model
!unzip -q /content/drive/MyDrive/Thesis/visual-sentiment-analysis-torch-models.zip -d .
!mkdir converted-models/
!cp /content/drive/MyDrive/Thesis/models/visual_models/paper_models/* converted-models/

hybrid_finetuned_all.pth   vgg19_finetuned_all.pth
hybrid_finetuned_fc6+.pth  vgg19_finetuned_fc6+.pth


In [ ]:
!python visual-sentiment-analysis-torch-models/predict.py test_set.txt --model vgg19_finetuned_fc6+ --batch-size 55 --root /content/drive/MyDrive/Thesis/dataset/benchmark/Twitter1269/twitter1/ > vgg19_finetuned_fc6+_predictions_at_least_5.csv

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0% 0/17 [00:00<?, ?it/s]/content/visual-sentiment-analysis-torch-models/vgg19.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax         = F.softmax(fc8_retrain_1)
100% 17/17 [00:59<00:00,  3.52s/it]


In [ ]:
# Predictions and accuracies on the Twitter Testing Dataset I with at least 5 agree
new_results_filtered = get_results(BENCHMARK_PREDICTION_DIR + 'at_least_5/', BENCHMARK_DIR + "5agree.csv", to_binary=True)
new_results_filtered

Gold labels {0: 301, 2: 581}
Pred labels {0: 401, 2: 481}
Pred labels {0: 409, 2: 473}
Pred labels {0: 378, 2: 504}
Pred labels {0: 391, 2: 491}


,Model,Accuracy
0,hybrid_finetuned_fc6+_predictions_at_least_5.csv,0.789116
1,hybrid_finetuned_all_predictions_at_least_5.csv,0.761905
2,vgg19_finetuned_all_predictions_at_least_5.csv,0.787982
3,vgg19_finetuned_fc6+_predictions_at_least_5.csv,0.764172


In [ ]:
# Predictions and accuracies on the Twitter Testing Dataset I with at least 4 agree
new_results_filtered = get_results(BENCHMARK_PREDICTION_DIR + 'at_least_4/', BENCHMARK_DIR + "4agree.csv", to_binary=True)
new_results_filtered

Gold labels {0: 427, 2: 689}
Pred labels {0: 528, 2: 588}
Pred labels {0: 531, 2: 585}
Pred labels {0: 555, 2: 561}
Pred labels {0: 560, 2: 556}


,Model,Accuracy
0,vgg19_finetuned_all_predictions_at_least_4.csv,0.766129
1,vgg19_finetuned_fc6+_predictions_at_least_4.csv,0.738351
2,hybrid_finetuned_all_predictions_at_least_4.csv,0.732975
3,hybrid_finetuned_fc6+_predictions_at_least_4.csv,0.758961


In [ ]:
# Predictions and accuracies on the Twitter Testing Dataset I with at least 5 agree
new_results_filtered = get_results(BENCHMARK_PREDICTION_DIR + 'at_least_3/', BENCHMARK_DIR + "3agree.csv", to_binary=True)
new_results_filtered

Gold labels {0: 500, 2: 769}
Pred labels {0: 628, 2: 641}
Pred labels {0: 624, 2: 645}
Pred labels {0: 659, 2: 610}
Pred labels {0: 664, 2: 605}


,Model,Accuracy
0,vgg19_finetuned_fc6+_predictions_at_least_3.csv,0.711584
1,vgg19_finetuned_all_predictions_at_least_3.csv,0.736801
2,hybrid_finetuned_all_predictions_at_least_3.csv,0.707644
3,hybrid_finetuned_fc6+_predictions_at_least_3 (...,0.735225


In [ ]:
# Accuracies on the B-T4SA 1.0 test set
original_results = get_results(ORIGINAL_PREDICTION_DIR, ORIGINAL_DATASET_DIR + 'b-t4sa_test.tsv')
original_results

,Model,Accuracy
0,vgg19_finetuned_all_predictions.csv,0.519206
1,vgg19_finetuned_fc6+_predictions.csv,0.507912
2,hybrid_finetuned_all_predictions.csv,0.504461
3,hybrid_finetuned_fc6+_predictions.csv,0.506226
